**Hey eveyone this is Parveshii and you are on the nb where we will fine tune your favourite gpt oss 20b or 120b**

GPT OSS is the open source model from OpenAI(ClosedAI) so as its a open source model we can fine tune it

<div class="markdown-google-sans">

## **Getting started**
</div>

we will be using:
1. Unsloth
2. trl
3. Transformers
these are main lib that we gonna use


In [ ]:
#lets download dependencies first
!pip install -U unsloth trl tranformers

86400

lets wrap the model in the unsloth fastlang wrapper


In [ ]:
from unsloth import FastLanguageModel
import torch

model, tok = FastLanguageModel(
    model_id = "openai/gpt-oss-20b",   # or "openai/gpt-oss-120b"
    max_seq_length = 2048,             # note: param name is max_seq_length
    dtype = torch.float16,             # use torch.float16 for fp16
    load_in_8bit = True,               # saves VRAM on T4
    device_map = "auto",
    full_finetuning = False            # keep False on T4, too small for full FT
)


604800

lets test the tokenizer

In [ ]:
t = tok.encode("hey bro")
m = tok.decode(t)
print(m)

**setup of lora**

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

#this will only work is full_finetuning = False as it doesn't train the whole model

#lets start to preprocess data

In [ ]:
from datasets import load_dataset, Dataset

# Load your dataset
dataset = load_dataset("Parveshiiii/opencode_reasoning_filtered")

# Example formatting function
def format_chat(example):
    messages = [
        {"role": "user", "content": example["input"]},#you cange this to the desired column you want to use as user or input whaih cvary in all dataset and its input in our case
        {"role": "assistant", "content": example["output"]}
    ]
    # Apply chat template → returns a single string in the model’s expected format
    formatted = tok.apply_chat_template(messages, tokenize=False)
    return {"text": formatted}

# Apply formatting
dataset_train = dataset.map(format_chat)

print(chat_ds["train"][0]["text"])


#lets setup trl


In [ ]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset_train,
    dataset_text_field="text",
    max_seq_length=4096,
    packing=True,
    args=SFTConfig(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        warmup_steps=10,
        max_steps=60,
        learning_rate=2e-5,
        logging_steps=5,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs_smol_instruct_math",
        report_to="none",
    ),
)

trainer.train()